In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input, Dropout, Activation, Dense, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import sklearn
from sklearn.preprocessing import MinMaxScaler

In [2]:
filepath = "iot23_combined.csv"
#df = pd.read_csv(filepath)
# df = pd.read_csv(r"C:\Users\Bas-Study\data\iot23_com\iot23_combined2.csv")
df = pd.read_csv(r"C:\Users\Bas-Study\data\df_combined.csv")

In [3]:
df.columns

Index(['Unnamed: 0', 'ts', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'duration', 'orig_bytes', 'resp_bytes', 'history', 'orig_pkts',
       'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', 'proto_icmp',
       'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ',
       'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR',
       'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2',
       'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR',
       'service_-', 'service_dhcp', 'service_dns', 'service_http',
       'service_irc', 'service_ssh', 'service_ssl'],
      dtype='object')

In [4]:
# del df['Unnamed: 0']
df.loc[(df.label == '-   Benign   -'), 'label'] = 'Benign'
df['label'].value_counts()
def drop_columns(df, columns):
    temp = df.copy()
    for column in columns:
        temp = temp[temp['label'] != column]
    return temp

columns_to_drop = [
                    "Attack",
                    "C&C-HeartBeat",
                    "C&C-FileDownload",
                    "C&C-Torii",
                    "FileDownload",
                    "C&C-HeartBeat-FileDownload",
                    "C&C-Mirai",
                    "PartOfAHorizontalPortScan"]

df = drop_columns(df, columns_to_drop)
df['label'].value_counts()

Okiru     262690
Benign    199756
DDoS      138777
C&C        15100
Name: label, dtype: int64

In [5]:
df['label'].value_counts()

Okiru     262690
Benign    199756
DDoS      138777
C&C        15100
Name: label, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]

In [56]:
#X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
X = df[['Unnamed: 0', 'ts','id.orig_p',
       'duration', 'orig_bytes', 'resp_bytes', 'orig_pkts',
       'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp',
       'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ',
       'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR',
       'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2',
       'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR',
       'service_-', 'service_dhcp', 'service_dns', 'service_http',
       'service_irc', 'service_ssh', 'service_ssl']]
# X.columns

In [57]:
X.shape

(616323, 33)

In [58]:
Y = pd.get_dummies(df['label']).values

In [59]:
Y.shape

(616323, 4)

In [11]:
#X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
#Y = df[['label']]

In [60]:
X

,Unnamed: 0,ts,id.orig_p,duration,orig_bytes,resp_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,...,conn_state_SF,conn_state_SH,conn_state_SHR,service_-,service_dhcp,service_dns,service_http,service_irc,service_ssh,service_ssl
3,3,1.525880e+09,34243.0,2.998804,0.0,0.0,3.0,180.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
4,4,1.525880e+09,34840.0,0.000000,0.0,0.0,1.0,60.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
14,14,1.525880e+09,34243.0,0.000000,0.0,0.0,1.0,60.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
27,27,1.525880e+09,34243.0,0.000000,0.0,0.0,1.0,60.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
44,44,1.525880e+09,33685.0,2.998560,0.0,0.0,3.0,180.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446616,122,1.562165e+09,68.0,-1.000000,-1.0,-1.0,11.0,3608.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1446617,123,1.562165e+09,143.0,-1.000000,-1.0,-1.0,9.0,844.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1446618,124,1.562165e+09,135.0,-1.000000,-1.0,-1.0,3.0,216.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1446619,125,1.562165e+09,143.0,-1.000000,-1.0,-1.0,8.0,648.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [13]:
df

,Unnamed: 0,ts,id.orig_h,id.orig_p,id.resp_h,id.resp_p,duration,orig_bytes,resp_bytes,history,...,conn_state_SF,conn_state_SH,conn_state_SHR,service_-,service_dhcp,service_dns,service_http,service_irc,service_ssh,service_ssl
3,3,1.525880e+09,192.168.100.103,34243.0,147.7.65.203,49560.0,2.998804,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
4,4,1.525880e+09,192.168.100.103,34840.0,145.164.35.6,21288.0,0.000000,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
14,14,1.525880e+09,192.168.100.103,34243.0,147.7.65.203,49560.0,0.000000,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
27,27,1.525880e+09,192.168.100.103,34243.0,147.7.65.203,49560.0,0.000000,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
44,44,1.525880e+09,192.168.100.103,33685.0,197.224.138.9,34797.0,2.998560,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446616,122,1.562165e+09,0.0.0.0,68.0,255.255.255.255,67.0,-1.000000,-1.0,-1.0,D,...,0,0,0,0,1,0,0,0,0,0
1446617,123,1.562165e+09,::,143.0,ff02::16,0.0,-1.000000,-1.0,-1.0,-,...,0,0,0,1,0,0,0,0,0,0
1446618,124,1.562165e+09,::,135.0,ff02::1:ffd5:cdf,136.0,-1.000000,-1.0,-1.0,-,...,0,0,0,1,0,0,0,0,0,0
1446619,125,1.562165e+09,fe80::5bcc:698e:39d5:cdf,143.0,ff02::16,0.0,-1.000000,-1.0,-1.0,-,...,0,0,0,1,0,0,0,0,0,0


In [37]:
# df_node = df[df['id.orig_h']=='192.168.100.111']
# df_node.shape
# df_node['label'].value_counts()


In [61]:
scaler = MinMaxScaler()

In [62]:
scaler.fit(X)

MinMaxScaler()

In [63]:
normalized_x = scaler.transform(X)

In [64]:
normalized_x

array([[3.00006000e-05, 0.00000000e+00, 5.22522660e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.00008000e-05, 0.00000000e+00, 5.31632435e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.40002800e-04, 0.00000000e+00, 5.22522660e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.24002480e-03, 8.41127626e-01, 2.05999939e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.25002500e-03, 8.41127626e-01, 2.18207343e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.26002520e-03, 8.41127626e-01, 2.02948088e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [65]:
normalized_x.shape

(616323, 33)

In [66]:
scaler.fit(Y)

MinMaxScaler()

In [67]:
normalized_y = scaler.transform(Y)

In [68]:
normalized_y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [69]:
#X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, test_size = .2, random_state=42)

In [70]:
X_train.shape

(493058, 33)

In [71]:
model = Sequential()

In [19]:
model.add(Dense(2000, activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(800,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [35]:
X_train.shape[1]


35

In [36]:
Y_train.shape[1]

4

In [72]:
model.add(Dense(64, activation='relu',input_dim=33))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [73]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                2176      
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dense_17 (Dense)            (None, 4)                 260       
                                                                 
Total params: 6,596
Trainable params: 6,596
Non-trainable params: 0
_________________________________________________________________


In [160]:
import time

In [39]:
Y_test.shape

(123265, 4)

In [74]:
# start = time.time()
# print('program start...')
# print()

model.fit(X_train, Y_train, epochs = 10, batch_size=256, validation_data=(X_test,Y_test),verbose=1)

# print()
# end = time.time()
# print('program end...')
# print()
# print('time cost: ')
# print(end - start, 'seconds')

Epoch 1/10
1927/1927 [==============================] - 9s 4ms/step - loss: 0.3395 - accuracy: 0.8721 - val_loss: 0.2694 - val_accuracy: 0.8884
Epoch 2/10
1927/1927 [==============================] - 7s 3ms/step - loss: 0.2463 - accuracy: 0.8959 - val_loss: 0.2419 - val_accuracy: 0.8796
Epoch 3/10
1927/1927 [==============================] - 7s 4ms/step - loss: 0.2274 - accuracy: 0.9084 - val_loss: 0.2184 - val_accuracy: 0.9331
Epoch 4/10
1927/1927 [==============================] - 7s 3ms/step - loss: 0.2132 - accuracy: 0.9189 - val_loss: 0.2232 - val_accuracy: 0.9123
Epoch 5/10
1927/1927 [==============================] - 8s 4ms/step - loss: 0.1991 - accuracy: 0.9254 - val_loss: 0.2048 - val_accuracy: 0.9005
Epoch 6/10
1927/1927 [==============================] - 7s 4ms/step - loss: 0.1819 - accuracy: 0.9339 - val_loss: 0.2430 - val_accuracy: 0.8663
Epoch 7/10
1927/1927 [==============================] - 8s 4ms/step - loss: 0.1661 - accuracy: 0.9431 - val_loss: 0.1565 - val_accuracy:

In [75]:
from sklearn.metrics import classification_report
a = np.argmax(Y_test, axis=1)
b = np.argmax(model.predict(X_test), axis=1)
print(classification_report(a, b))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92     39812
           1       0.91      0.91      0.91      3016
           2       1.00      1.00      1.00     28013
           3       0.94      0.95      0.94     52424

    accuracy                           0.95    123265
   macro avg       0.94      0.94      0.94    123265
weighted avg       0.95      0.95      0.95    123265



In [76]:
preds = np.argmax(model.predict(X_test), axis = 1)
num_correct = 0

for i in range(0, len(preds)):
    if preds[i] == np.argmax(Y_test[i]):
        num_correct+=1

num_correct/len(preds)

0.9479089765951405